<font size="7">GENERATING PREDICTION FILE</font>

In [23]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import joblib

ARTIFACTS = Path("artifacts")
DATA_DIR = Path("data")
OUTPUT_FILE = "Piyush_Sagatani_23115104_Submission.csv"

print("Artifacts dir:", ARTIFACTS.resolve())
print("Data dir:", DATA_DIR.resolve())

Artifacts dir: C:\Users\piyus\OneDrive\Desktop\Project\artifacts
Data dir: C:\Users\piyus\OneDrive\Desktop\Project\data


In [ ]:
df_test = pd.read_csv("test2(test(1)).csv")
df_test = df_test.drop_duplicates('id')

print("Test rows:", len(df_test))
df_test

Test rows: 5396


,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5399,7732500270,20140925T000000,4,2.50,2820,15000,2.0,0,0,4,9,2820,0,1985,0,98052,47.7255,-122.101,2440,15000
5400,3856903515,20141222T000000,3,2.00,1460,6250,1.5,0,0,4,7,1460,0,1912,0,98103,47.6693,-122.333,1690,4750
5401,2557000400,20150409T000000,3,2.50,2070,9900,1.0,0,0,3,8,1420,650,1979,0,98023,47.2988,-122.371,2070,8250
5402,4386700135,20141114T000000,4,2.25,4760,8036,2.5,0,0,5,9,3390,1370,1916,0,98112,47.6415,-122.285,2950,9323


In [ ]:
img_feat_path = ARTIFACTS / "test_image_features_resnet.csv"
img_feats = pd.read_csv(img_feat_path)

img_feats.columns = [c.lower().strip() for c in img_feats.columns]
print("Image features rows:", len(img_feats), "columns:", len(img_feats.columns))

df_test_merge = df_test.merge(img_feats, on="id", how="left")
print("After merge shape:", df_test_merge.shape)

feat_cols_img = [c for c in df_test_merge.columns if c.startswith("img_") or c.startswith("emb_")]
print("Found image embedding columns:", len(feat_cols_img))

if len(feat_cols_img) == 0:
    raise ValueError("No image embedding columns found in image_features_resnet.csv. Check extraction output filename/columns.")

df_test_merge[feat_cols_img] = df_test_merge[feat_cols_img].fillna(0.0)
display(df_test_merge)

Image features rows: 5404 columns: 513
After merge shape: (5404, 532)
Found image embedding columns: 512


,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,img_502,img_503,img_504,img_505,img_506,img_507,img_508,img_509,img_510,img_511
0,2591820310,20141006T000000,4,2.25,2070,8893,2.0,0,0,4,...,0.527963,1.366555,0.953507,0.520843,0.209390,1.649956,2.982348,2.193365,0.032101,1.839500
1,7974200820,20140821T000000,5,3.00,2900,6730,1.0,0,0,5,...,0.262981,0.934756,0.512699,1.492166,0.226773,1.269342,0.934164,3.011106,0.013243,0.865530
2,7701450110,20140815T000000,4,2.50,3770,10893,2.0,0,2,3,...,0.359296,1.304909,0.817359,0.780644,0.115067,0.456621,0.827820,0.120380,0.000821,0.040966
3,9522300010,20150331T000000,3,3.50,4560,14608,2.0,0,2,3,...,0.169283,0.886541,0.093300,0.979192,0.038776,0.936102,0.071240,0.488793,0.002306,0.733583
4,9510861140,20140714T000000,3,2.50,2550,5376,2.0,0,0,3,...,0.149116,2.392144,0.336435,1.230908,1.133957,0.214105,1.230468,0.620270,0.000000,0.048085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5399,7732500270,20140925T000000,4,2.50,2820,15000,2.0,0,0,4,...,0.072083,0.831319,0.380476,1.303221,0.072768,0.932203,1.187632,2.459741,0.066766,0.076514
5400,3856903515,20141222T000000,3,2.00,1460,6250,1.5,0,0,4,...,0.302908,2.223312,0.044965,0.627866,0.511835,1.599862,2.578853,0.950357,0.004185,0.099375
5401,2557000400,20150409T000000,3,2.50,2070,9900,1.0,0,0,3,...,0.267890,0.767162,0.111298,0.381230,0.443771,0.528057,1.303191,1.236690,0.015115,0.024518
5402,4386700135,20141114T000000,4,2.25,4760,8036,2.5,0,0,5,...,0.412594,0.894101,0.064491,1.440025,0.137948,0.494031,0.028053,2.661992,0.009393,0.091615


In [ ]:
train_processed_path = DATA_DIR / "processed" / "df_train_eng.csv"

train_proc = pd.read_csv(train_processed_path)

train_merged = train_proc.merge(img_feats, on="id", how="inner")
print("Train merged shape:", train_merged.shape)


feature_cols_order = [c for c in train_merged.columns if c not in ("id", "price")]
print("Number of features used in training:", len(feature_cols_order))
print("First 20 features:", feature_cols_order[:20])

Train merged shape: (71, 548)
Number of features used in training: 546
First 20 features: ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6']


In [ ]:
for c in feature_cols_order:
    if c not in df_test_merge.columns:
        df_test_merge[c] = 0.0

X_test = df_test_merge[feature_cols_order].copy()
print("X_test shape:", X_test.shape)

X_test shape: (5404, 546)


C:\Users\piyus\AppData\Local\Temp\ipykernel_141224\1368548560.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_merge[c] = 0.0
C:\Users\piyus\AppData\Local\Temp\ipykernel_141224\1368548560.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_merge[c] = 0.0
C:\Users\piyus\AppData\Local\Temp\ipykernel_141224\1368548560.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

In [ ]:
best_model_path = ARTIFACTS / "best_model.pkl"

pipeline = joblib.load(best_model_path)
print("Loaded pipeline:", pipeline)

X_test_np = X_test.values.astype(float)
preds = pipeline.predict(X_test_np)

Loaded pipeline: Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.8, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                           

c:\Users\piyus\miniconda3\envs\Img\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
df_output = pd.DataFrame({
    "id": df_test_merge["id"].astype("int64"),
    "predicted_price": np.round(preds.astype(float), 5)
})


df_output = df_output[["id", "predicted_price"]]
df_output = df_output.drop_duplicates(subset=['id'], keep='first')


df_output.to_csv(OUTPUT_FILE, index=False, float_format="%.5f")
print(f"Saved submission to {OUTPUT_FILE} ({len(df_output)} rows)")


df_output.head(10)

Saved submission to Piyush_Sagatani_23115104_Submission.csv (5396 rows)


,id,predicted_price
0,2591820310,3.846516e+05
1,7974200820,6.008102e+05
2,7701450110,1.624948e+06
3,9522300010,2.083218e+06
4,9510861140,6.989112e+05
5,1761300310,2.772469e+05
6,7732410120,7.083050e+05
7,7010701383,4.137054e+05
8,291310170,3.252717e+05
9,4232901990,3.817543e+05
